reading the table

In [0]:
 from pyspark.sql.functions import *
 from pyspark.sql.types import *

In [0]:
nyc_dataset=spark.read.table('tester.default.bronze_data')

In [0]:
nyc_dataset=nyc_dataset.drop("_rescued_data")

null checker

In [0]:
null_count=nyc_dataset.select([sum(col(c).isNull().cast("int")).alias(c)  for c in nyc_dataset.columns])

In [0]:
display(null_count)

In [0]:
nyc_dataset=nyc_dataset.drop('rate_code_id','store_and_fwd_flag')

In [0]:
nyc_dataset=nyc_dataset.fillna(0,subset=['mta_tax','total_amount'])

In [0]:
nyc_dataset=nyc_dataset.withColumn('pickup_date',to_date(col("pickup_datetime")))\
    .withColumn('dropoff_date',to_date(col("dropoff_datetime")))\
        .withColumn('pickup_time',date_format(col('pickup_datetime'),"HH:mm:ss"))\
            .withColumn('dropoff_time',date_format(col("dropoff_datetime"),"HH:mm:ss"))\
                

### validations


In [0]:
valid_trip_1=nyc_dataset.filter(col('pickup_datetime')<col("dropoff_datetime"))
print("Valid trip count with datetime filtration"+str(valid_trip_1.count()))

In [0]:
valid_trip_1.groupBy('payment_type').agg(count('*').alias("count")).display()

In [0]:
valid_trip_2=valid_trip_1.filter((col('passenger_count')!=0)&(col('trip_distance')>=0))

In [0]:
print("Valid trip count with passenger trip  iltration : " + str(valid_trip_2.count()))

In [0]:
valid_trip_3=valid_trip_2.filter((col('fare_amount')>=0) & (col('total_amount')>=0)&(col('total_amount')>=col('fare_amount')+col('tolls_amount')+col('tip_amount')+col('mta_tax')+col('extra')))

In [0]:
print("Valid trips with  fare amount  filtration :"+str(valid_trip_3.count()))

In [0]:
valid_trip_4=valid_trip_3.drop_duplicates()

In [0]:
print("Valid rows final : "+str(valid_trip_4.count()))

In [0]:

(
    valid_trip_4.write
    .format("delta")
    .mode("append")                 # or "overwrite"
    .option("mergeSchema", "true")  # if you expect new columns
    .saveAsTable("tester.default.silver_data")
)

        